In [1]:
import pyomo.environ as pyo

# Parameters
N = 50  # Number of time steps
dt = 0.1  # Time step size
x0 = [0, 0]  # Initial state: position and velocity
xf = [10, 0]  # Final state: position and velocity

# Create a Pyomo model
model = pyo.ConcreteModel()

# Time
model.t = pyo.RangeSet(0, N)

# Decision variables
model.x1 = pyo.Var(model.t)  # Position
model.x2 = pyo.Var(model.t)  # Velocity
model.u = pyo.Var(model.t)  # Control

# Initial and final conditions
model.constraints = pyo.ConstraintList()
model.constraints.add(model.x1[0] == x0[0])
model.constraints.add(model.x2[0] == x0[1])
model.constraints.add(model.x1[N] == xf[0])
model.constraints.add(model.x2[N] == xf[1])

# Dynamics (Euler discretization)
for i in model.t:
    if i < N:  # Skip the last point
        model.constraints.add(model.x1[i+1] == model.x1[i] + dt * model.x2[i])
        model.constraints.add(model.x2[i+1] == model.x2[i] + dt * model.u[i])

# Objective: Minimize the control effort
def objective_rule(model):
    return sum(model.u[i]**2 for i in model.t)

model.objective = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Solve the problem
solver = pyo.SolverFactory('ipopt')
results = solver.solve(model, tee=True)

# Extract the solution
x1_opt = [pyo.value(model.x1[i]) for i in model.t]
x2_opt = [pyo.value(model.x2[i]) for i in model.t]
u_opt = [pyo.value(model.u[i]) for i in model.t]

# You can now use x1_opt, x2_opt, and u_opt for further analysis


Ipopt 3.11.1: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

NOTE: You are using Ipopt by default with the MUMPS linear solver.
      Other linear solvers might be more efficient (see Ipopt documentation).


This is Ipopt version 3.11.1, running with linear solver mumps.

Number of nonzeros in equality constraint Jacobian...:      304
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       51

Total number of variables............................:      153
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0


In [2]:
import torch
import torch.nn as nn

class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc = nn.Linear(1, 1)  # A single linear layer

    def forward(self, x):
        return self.fc(x)

# Initialize the model
nn_model = SimpleNN()


In [3]:
import pyomo.environ as pyo

# Create a Pyomo model
pyomo_model = pyo.ConcreteModel()

# Define a design variable
pyomo_model.x = pyo.Var(initialize=0.5, within=pyo.Reals)

# Define a PyTorch-based objective function
def pytorch_objective(model):
    x_value = pyo.value(model.x)
    x_tensor = torch.tensor([x_value], dtype=torch.float32)

    # Forward pass through the NN
    y_tensor = nn_model(x_tensor)
    
    return y_tensor.item()

# Set the objective
pyomo_model.objective = pyo.Objective(rule=pytorch_objective, sense=pyo.minimize)


In [4]:
# Solve the problem using IPOPT
solver = pyo.SolverFactory('ipopt')
results = solver.solve(pyomo_model, tee=True)

# Output the results
print(f"Optimal x: {pyo.value(pyomo_model.x)}")


ValueError: No variables appear in the Pyomo model constraints or objective. This is not supported by the NL file interface

In [5]:
import numpy as np
import pyomo.environ as pyo
import torch
import torch.nn as nn

# Define your PyTorch model
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc = nn.Linear(1, 1)

    def forward(self, x):
        return self.fc(x)

nn_model = SimpleNN()

# Example function to integrate PyTorch and Pyomo
def integrate_pytorch_pyomo(x):
    # Convert x to a tensor
    x_tensor = torch.tensor([x], dtype=torch.float32)

    # Forward pass through the PyTorch model
    y_tensor = nn_model(x_tensor)

    # Convert the PyTorch output to a numpy value
    return y_tensor.detach().numpy()[0]

# Create a Pyomo model
pyomo_model = pyo.ConcreteModel()

# Define a design variable
pyomo_model.x = pyo.Var(initialize=0.5, within=pyo.Reals)

# Define the objective function using Pyomo variables and the PyTorch model
def pyomo_objective(model):
    # Use the PyTorch model within the Pyomo objective
    pytorch_output = integrate_pytorch_pyomo(pyo.value(model.x))
    return pytorch_output

# Set the objective
pyomo_model.objective = pyo.Objective(rule=pyomo_objective, sense=pyo.minimize)

# Solve the problem using IPOPT
solver = pyo.SolverFactory('ipopt')
results = solver.solve(pyomo_model, tee=True)

# Output the results
print(f"Optimal x: {pyo.value(pyomo_model.x)}")


ValueError: No variables appear in the Pyomo model constraints or objective. This is not supported by the NL file interface